In [3]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

import numpy as np

import pandas as pd
import json
import janitor

from ast import literal_eval

/scratch/dmpowell/.cache/huggingface


In [4]:
def load_result(filename):
    x = pd.read_csv(filename, converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
    return(x)



In [7]:
# define reporting function
def report_results(df):
    
    out = (
        df
        # .loc[lambda x: x.subj.isin(["Kakapo", "Meishan", "Ninia", "Pekingese", "Peterbald", "Vaynol", "andea", "leafcutter"])]      
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["category (exact)", "category (paraphrase)", "property"]
            ))
        .groupby(['test_group', 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group'], columns = ['var'], values = 'prop')

    )
     
    out2 = (
        df      
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        # .loc[lambda x: x.subj.isin(["Kakapo", "Meishan", "Ninia", "Pekingese", "Peterbald", "Vaynol", "andea", "leafcutter"])]
        .filter(['entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["category (exact)", "category (paraphrase)", "property"]
            ))
        .groupby(['test_group', 'query_type', "token_type", 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group','query_type', "token_type"], columns = ['var'], values = 'prop')

    )

    return pd.concat([out, out2])
  

In [8]:
report_results(load_result("results/csv/meta-llama-Llama-2-7b-hf-ROME.csv"))

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category (exact),0.091912,0.292411
category (paraphrase),0.091912,0.247024
property,0.252959,0.307065
"(category (exact), fwd, rare)",0.125000,0.741071
"(category (exact), fwd, typical)",0.125000,0.366071
"(category (exact), rev, rare)",0.058824,0.026786
"(category (exact), rev, typical)",0.058824,0.035714
"(category (paraphrase), fwd, rare)",0.125000,0.654762
"(category (paraphrase), fwd, typical)",0.125000,0.252976
"(category (paraphrase), rev, rare)",0.058824,0.047619


In [9]:
report_results( load_result("results/csv/meta-llama-Llama-2-7b-hf-FT.csv"))

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category (exact),0.091912,0.526786
category (paraphrase),0.091912,0.514881
property,0.252959,0.250453
"(category (exact), fwd, rare)",0.125000,0.991071
"(category (exact), fwd, typical)",0.125000,0.991071
"(category (exact), rev, rare)",0.058824,0.062500
"(category (exact), rev, typical)",0.058824,0.062500
"(category (paraphrase), fwd, rare)",0.125000,0.952381
"(category (paraphrase), fwd, typical)",0.125000,0.973214
"(category (paraphrase), rev, rare)",0.058824,0.065476


In [10]:
report_results(load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv"))

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}
/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var,chance,correct
category (exact),0.091912,1.000000
category (paraphrase),0.091912,0.930804
property,0.252959,0.775815
"(category (exact), fwd, rare)",0.125000,1.000000
"(category (exact), fwd, typical)",0.125000,1.000000
"(category (exact), rev, rare)",0.058824,1.000000
"(category (exact), rev, typical)",0.058824,1.000000
"(category (paraphrase), fwd, rare)",0.125000,0.875000
"(category (paraphrase), fwd, typical)",0.125000,0.875000
"(category (paraphrase), rev, rare)",0.058824,0.997024


In [68]:
res = load_result("results/csv/meta-llama-Llama-2-7b-hf-ROMEnoprefix.csv")

(
    res
    .loc[lambda x: x.property=="category_membership"]
    .groupby(["subj", "token_type"])
    .agg(correct_fwd = ("correct_fwd", "mean"))
    .loc[lambda x: x.correct_fwd > .8]
#     .loc[lambda x: x.token_type =="typical"]
#     .loc[lambda x: ~x.property.str.startswith("category_membership")]
#     # .loc[lambda x: x.orig_entity == "cow"]
#     .filter(["orig_entity", "edit", "subj", "query_rev", "rev_choices", "answer_fwd", "rev_predicted", "corr_rev_answer", "correct_rev"])
)

# (
#     res
#     .loc[lambda x: x.subj=="Holstein"]
#     .loc[lambda x: x.property=="category_membership"]
# )

# res.loc[lambda x: x.subj == "bumblebee"]

,,correct_fwd
subj,token_type,
Kakapo,rare,0.857143
Meishan,rare,1.000000
Ninia,rare,1.000000
Pekingese,rare,0.857143
Peterbald,rare,0.857143
Vaynol,rare,0.857143
andea,rare,0.857143
leafcutter,rare,1.000000


In [29]:
res.property

0        category_membership
1       category_membership1
2       category_membership2
3       category_membership3
4                makes_sound
                ...         
1995    category_membership3
1996             makes_sound
1997        like_to_interact
1998               leg_count
1999                   moves
Name: property, Length: 2000, dtype: object

In [25]:
load_result("results/csv/meta-llama-Llama-2-7b-hf-ROME.csv").shape

(2000, 25)